Mount goole drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir('/content/drive/My Drive/Major Project/DataSet')
os.getcwd()

'/content/drive/My Drive/Major Project/DataSet'

Including Necessary Libraries

In [ ]:
import os
import pandas
import numpy


import sklearn.ensemble as ek
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix , f1_score , accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.under_sampling import NearMiss
import imblearn
from mlxtend.classifier import StackingCVClassifier 

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functio

Load DataSet

In [ ]:
dataset = pandas.read_csv('Tobin12_Missing_Value.csv',sep=',') #type 3
#dataset = dataset.drop(['TobinsQ'], axis = 1) #type 2
#dataset = dataset.drop(['MarketBookRatio'],axis = 1) #type 4
#dataset = dataset.drop(['TobinsQ','OperationalMargin','ReturnOnEquity','MarketBookRatio','AssetsGrowth','SalesGrowth','EmployeeGrowth'],axis = 1) #type 1

In [ ]:
dataset.head()

,TobinsQ,Liquidity,Profitability,Productivity,LeverageRatio,AssetTurnover,OperationalMargin,ReturnOnEquity,MarketBookRatio,AssetsGrowth,SalesGrowth,EmployeeGrowth,BK
0,0.98,0.36,0.18,0.13,1.33,1.77,0.07,0.15,2.22,NaN,NaN,NaN,0
1,0.98,0.36,0.19,0.12,1.31,1.59,0.07,0.13,2.41,0.126,0.014,0.040,0
2,0.87,0.32,0.13,0.08,1.03,1.55,0.05,0.04,2.56,0.368,0.328,0.567,0
3,1.13,0.28,0.14,0.08,0.80,1.39,0.06,0.05,5.28,-0.021,-0.119,-0.096,0
4,1.26,0.41,0.13,0.08,0.20,1.30,0.06,0.04,8.68,0.233,0.147,0.053,0


In [ ]:
X = dataset.drop(['BK'],axis=1).values
Y = dataset['BK'].values

Missing Value handling using KNN Imputation

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(n_neighbors=15, weights='uniform', metric='nan_euclidean')

In [ ]:
imputer.fit(X)

KNNImputer(add_indicator=False, copy=True, metric='nan_euclidean',
           missing_values=nan, n_neighbors=15, weights='uniform')

In [ ]:
Xtrans = imputer.transform(X)

In [ ]:
from numpy import isnan
print('Missing: %d' % sum(isnan(Xtrans).flatten()))

Missing: 0


Downsampling to balance the data set using NearMiss algorithm

In [ ]:
nm = NearMiss(random_state=42)
x,y = nm.fit_sample(Xtrans,Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
x.shape , y.shape

((1116, 12), (1116,))

Test Train Split of the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y ,test_size=0.2)

Base algorithms for Bi-level classification technique

In [ ]:
classifier1 = ek.BaggingClassifier(base_estimator=ek.RandomForestClassifier(n_estimators=50),n_estimators=10, random_state=0)
classifier2 = ek.AdaBoostClassifier(n_estimators=50)
classifier3 = ek.BaggingClassifier(base_estimator=ek.AdaBoostClassifier(n_estimators=50),n_estimators=10, random_state=0)
classifier4 = tree.DecisionTreeClassifier(max_depth=10)
classifier5 = svm.SVC(kernel = 'linear')
classifier6 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,random_state=0)

Making Stack for Bi-level classification technique.

In [ ]:
sclf = StackingCVClassifier(classifiers = [classifier1, classifier2,classifier3,classifier4,classifier5],
                            shuffle = False,
                            use_probas = False,
                            
                            meta_classifier  = ek.RandomForestClassifier(n_estimators=50) )

In [ ]:
classifiers = {"BaggingR": classifier1,
               "AdaBoost": classifier2,
               "BaggingA": classifier3,
               "Decision Tree": classifier4,
               "Linear SVM": classifier5,
               "Gradient Boosting Classifier": classifier6,
               "Random Forest": sclf}

Training of Model

In [ ]:
for key in classifiers:
    # Get classifier
    classifier = classifiers[key]
    
    # Fit classifier
    classifier.fit(X_train, y_train)
        
    # Save fitted classifier
    classifiers[key] = classifier

Testing of Model

In [ ]:
for key in classifiers:
    # Make prediction on test set
    y_pred = classifiers[key].predict(X_test)

Evaluation of Model

In [ ]:
F1 = f1_score(y_test, y_pred, average='weighted')
Accuracy = accuracy_score(y_test,y_pred)
print("F1: ", F1)
print("Accuracy: ",Accuracy)

F1:  0.9732142857142857
Accuracy:  0.9732142857142857
